<a href="https://colab.research.google.com/github/harjeet88/agentic_ai/blob/main/crew_ai/financial_stock_adviser_infinite_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install --quiet crewai crewai_tools langchain openai langchain_community

  Using cached crewai-0.130.0-py3-none-any.whl.metadata (35 kB)
  Using cached crewai_tools-0.47.1-py3-none-any.whl.metadata (10 kB)
  Using cached langchain_community-0.3.26-py3-none-any.whl.metadata (2.9 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 324.6/324.6 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.0/8.0 MB 67.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 81.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 606.3/606.3 kB 38.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━

In [11]:
!pip install --quiet langgraph langchain_openai gradio python-dotenv pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 15.4 MB/s eta 0:00:00


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
from typing import Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from dotenv import load_dotenv
from IPython.display import Image, display
import gradio as gr
from langgraph.graph import StateGraph
from langgraph.graph.message import add_messages
from langchain_openai import ChatOpenAI
from pydantic import BaseModel
import random
import os

In [14]:
load_dotenv('/content/drive/MyDrive/collab/.env')

True

In [15]:
from crewai_tools import SerperDevTool
import os
from crewai import LLM,Agent, Task, Crew
from langchain.llms import OpenAI
from google.colab import userdata

In [16]:
gemini_flash_25="gemini/gemini-2.5-flash"

In [17]:
selected_model=gemini_flash_25

In [18]:
llm = LLM(
    api_key=os.getenv("GEMINI_API_KEY"),
    model=selected_model,
)

In [36]:
research_agent = Agent(
    llm = llm,
    role="Senior Financial Researcher for {company}",
    goal="Research the company, news and potential for {company}",
    backstory="""You're a seasoned financial researcher with a talent for finding
    the most relevant information about {company}.
    Known for your ability to find the most relevant
    information and present it in a clear and concise manner.""",
    #tools=[SerperDevTool()],
    verbose=True  # Enable logging for debugging
)

In [37]:
analyst_agent= Agent(
    llm = llm,
    role="Market Analyst and Report writer focused on {company}",
    goal=""" Analyze company {company} and create a comprehensive, well-structured report
    that presents insights in a clear and engaging way""",
    backstory=""" You're a meticulous, skilled analyst with a background in financial analysis
    and company research. You have a talent for identifying patterns and extracting
    meaningful insights from research data, then communicating
    those insights through well crafted reports""",
    tools=[SerperDevTool()],
    verbose=True  # Enable logging for debugging
)

In [38]:
research_task = Task(
    description="""Conduct thorough research on company {company}. Focus on:
    1. Current company status and health
    2. Historical company performance
    3. Major challenges and opportunities
    4. Recent news and events
    5. Future outlook and potential developments

    Make sure to organize your findings. donot use more 5 searches """,
    expected_output=""" A comprehensive research document with well-organized sections covering
    all the requested aspects of {company}. Include specific facts, figures,
    and examples where relevant.""",
    agent=research_agent
)

In [39]:
analysis_task = Task(
    description="""Analyze the research findings and create a comprehensive report on {company}.
    Your report should:
    1. Begin with an executive summary
    2. Include all key information from the research
    3. Provide insightful analysis of trends and patterns
    4. Offer a market outlook for company, noting that this should not be used for trading decisions
    5. Be formatted in a professional, easy-to-read style with clear headings""",
    expected_output="""  A polished, professional report on {company} that presents the research
    findings with added analysis and insights. The report should be well-structured
    with an executive summary, main sections, and conclusion.""",
    agent=analyst_agent
)

In [40]:
# Define Crew
crew = Crew(
    agents=[research_agent,analyst_agent],
    tasks=[research_task,analysis_task],
    verbose=True
)

In [41]:
data = {
        'company': 'Apple'
    }
result = crew.kickoff(inputs=data)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 6147d213-0ee5-435c-a4cc-ff0693b6a459                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

# Agent: Senior Financial Researcher for Apple

## Task: Conduct thorough research on company Apple. Focus on:
    1. Current company status and health
    2. Historical company performance
    3. Major challenges and opportunities
    4. Recent news and events
    5. Future outlook and potential developments

    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure

## Task: Conduct thorough research on company Apple. Focus on:
    1. Current company status and health
    2. Historical company performance
    3. Major challenges and opportunities
    4. Recent news and events
    5. Future outlook and potential developments

    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure

Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 sear

## Task: Conduct thorough research on company Apple. Focus on:
    1. Current company status and health
    2. Historical company performance
    3. Major challenges and opportunities
    4. Recent news and events
    5. Future outlook and potential developments

    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure to organize your findings. donot use more 5 searches [00m
    Make sure

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 6147d213-0ee5-435c-a4cc-ff0693b6a459                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RecursionError: maximum recursion depth exceeded in comparison

RecursionError: maximum recursion depth exceeded